## PyConsult

#### PiConsult functionality in a desktop form using Tkinter

In [119]:
import tkinter as tk
import tkinter.ttk as ttk

import numpy as np
import serial

In [121]:
def launch_data_stream():
    global landing
    landing.destroy() # close original window
    
    datastream = tk.Tk()
    datastream.title("Data Stream")
    # code to do the datastream

def launch_dtc():
    global landing
    landing.destroy() # close original window
    
    dtc = tk.Tk()
    dtc.title("DTCs")
    # code to do the datastream

def launch_testing():
    global landing
    landing.destroy() # close original window
    
    testing = tk.Tk()
    testing.title("Testing Mode")
    


def launch_settings():
    global landing
    landing.destroy() # close original window
    
    settings = tk.Tk()
    settings.title("Settings")
    settings.focus_force()



In [195]:
### Main landing/startup window ###
# GUI parameters
ipadx = 40
ipady = 40
padx = 10
pady = 10
geox = 460
geoy = 280
buttonwidth = 20

# Create main window
landing = tk.Tk()
landing.title("PyConsult GUI")
landing.geometry(str(geox) + "x" + str(geoy))
landing.resizable(False,False)
landing.focus_force()

# Create buttons and labels
selectionlabel = ttk.Label(landing, text="Select Mode!",width=10,font=("Arial", 16))
ds_button = ttk.Button(landing, text="Data Stream",width=buttonwidth,command=launch_data_stream)
dtc_button = ttk.Button(landing, text="DTCs",width=buttonwidth,command=launch_dtc)
test_button = ttk.Button(landing, text="Testing",width=buttonwidth,command=launch_testing)
setting_button = ttk.Button(landing, text="Settings",width=buttonwidth,command=launch_settings)

ds_button.grid(row=1,column=0, ipadx=ipadx, ipady=ipady,padx=padx, pady=pady) 
dtc_button.grid(row=2,column=0, ipadx=ipadx, ipady=ipady,padx=padx, pady=pady) 
test_button.grid(row=1,column=1, ipadx=ipadx, ipady=ipady,padx=padx, pady=pady) 
setting_button.grid(row=2,column=1, ipadx=ipadx, ipady=ipady,padx=padx, pady=pady) 

selectionlabel.grid(row=0,column=0,columnspan=2)

landing.mainloop()


In [196]:
### Settings Window ###

# GUI parameters
ipadx = 40
ipady = 40
padx = 10
pady = 10
geox = 460
geoy = 280
buttonwidth = 20

# Create settings window
settings = tk.Tk()
settings.title("PyConsult Settings")
settings.geometry(str(geox) + "x" + str(geoy))
settings.resizable(False,False)
settings.focus_force()

# Create input boxes for settings
rpm_warning = ttk.Entry(settings, text="RPM Warning")
temp_warning = ttk.Entry(settings, text="Temperature Warning")

# Create labels for input boxes
rpm_warning_label = ttk.Label(settings, text="RPM Warning")
temp_warning_label = ttk.Label(settings, text="Temperature Warning")

# Place input boxes and labels in grid
rpm_warning.grid(row=0,column=1, padx=padx, pady=pady)
rpm_warning_label.grid(row=0,column=0, padx=padx, pady=pady)
temp_warning.grid(row=1,column=1, padx=padx, pady=pady)
temp_warning_label.grid(row=1,column=0, padx=padx, pady=pady)

settings.mainloop()
